In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd

In [4]:
import sys
sys.path = ['../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, smiles2graph, BERTize

In [5]:
from dfs_transformer import DFSCodeSeq2SeqFCFeatures, Trainer, PubChem, get_n_files
from dfs_transformer.training.utils import seq_loss, seq_acc, collate_BERT, collate_rnd2min
import argparse
import yaml
import functools
from ml_collections import ConfigDict
fname = '../../config/selfattn/bert10M.yaml'

In [6]:
with open(fname) as file:
    config = ConfigDict(yaml.load(file, Loader=yaml.FullLoader))

In [7]:
name = "fbert-10M"
mode = "online"

In [8]:
run = wandb.init(mode=mode, project="pubchem-experimental", entity="chrisxx", 
                 name=name, config=config.to_dict())

wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-09-16 08:53:37.080951: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-16 08:53:37.080982: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
m = config.model
t = config.training
d = config.data
device = torch.device('cuda:%d'%config.training.gpu_id if torch.cuda.is_available() else 'cpu')

ce = nn.CrossEntropyLoss(ignore_index=-1)
bce = nn.BCEWithLogitsLoss()    

fields = ['acc-dfs1', 'acc-dfs2', 'acc-atm1', 'acc-atm2', 'acc-bnd']
metrics = {field:functools.partial(seq_acc, idx=idx) for idx, field in enumerate(fields)}

In [10]:
def collate_fn(dlist, fraction_missing=t.fraction_missing):
    node_batch = [] 
    edge_batch = []
    min_code_batch = []
    for d in dlist:
        node_batch += [d.node_features]
        edge_batch += [d.edge_features]
        atom1 = d.node_features[d.min_dfs_code[:, -3]]
        atom2 = d.node_features[d.min_dfs_code[:, -1]]
        bond = d.edge_features[d.min_dfs_code[:, -2]]
        min_code_batch += [torch.cat((d.min_dfs_code, atom1, atom2, bond), dim=1)]

    inputs, outputs = BERTize(min_code_batch, fraction_missing=fraction_missing)
    inputs = [inp[:, :8].long() for inp in inputs]
    targets = nn.utils.rnn.pad_sequence(outputs, padding_value=-1)
    return inputs, node_batch, edge_batch, targets 

In [11]:
def loss(pred, target):
    dfs1, dfs2, atm1, atm2, bnd, feat = pred
    
    pred_dfs1 = torch.reshape(dfs1, (-1, m.max_nodes))
    pred_dfs2 = torch.reshape(dfs2, (-1, m.max_nodes))
    pred_atm1 = torch.reshape(atm1, (-1, m.n_atoms))
    pred_atm2 = torch.reshape(atm2, (-1, m.n_atoms))
    pred_bnd = torch.reshape(bnd, (-1, m.n_bonds))
    pred_feat = torch.reshape(feat, (-1,  2*m.n_node_features + m.n_edge_features))
    
    tgt_dfs1 = target[:, :, 0].view(-1).long()
    tgt_dfs2 = target[:, :, 1].view(-1).long()
    tgt_atm1 = target[:, :, 2].view(-1).long()
    tgt_atm2 = target[:, :, 4].view(-1).long()
    tgt_bnd = target[:, :, 3].view(-1).long()
    tgt_feat = target[:, :, 8:].view(-1, 2*m.n_node_features + m.n_edge_features)
    
    loss = ce(pred_dfs1, tgt_dfs1) 
    loss += ce(pred_dfs2, tgt_dfs2)
    loss += ce(pred_atm1, tgt_atm1)
    loss += ce(pred_bnd, tgt_bnd)
    loss += ce(pred_atm2, tgt_atm2)
    
    mask = tgt_dfs1 != -1
    loss += bce(pred_feat[mask], tgt_feat[mask])
    
    return loss 

In [12]:
model = DFSCodeSeq2SeqFCFeatures(**m)
    
if t.load_last and t.es_path is not None:
    model.load_state_dict(torch.load(t.es_path, map_location=device))
elif t.pretrained_dir is not None:
    model.load_state_dict(torch.load(t.pretrained_dir, map_location=device))

In [13]:
validloader = None
if d.valid_path is not None:
    validset = PubChem(d.valid_path, max_nodes=m.max_nodes, max_edges=m.max_edges)
    validloader = DataLoader(validset, batch_size=d.batch_size, shuffle=True, 
                             pin_memory=False, collate_fn=collate_fn)
    exclude = validset.smiles

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9978/9978 [00:00<00:00, 10443.75it/s]


In [14]:
data = next(iter(validloader))

In [15]:
data[-1][1]

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

In [16]:
trainer = Trainer(model, None, loss, validloader=validloader, metrics=metrics, 
                  wandb_run = run, **t)
trainer.n_epochs = d.n_iter_per_split

In [17]:
n_files = get_n_files(d.path)
if d.n_used is None:
    n_splits = 1
else:
    n_splits = n_files // d.n_used

In [ ]:
for epoch in range(t.n_epochs):
    print('starting epoch %d'%(epoch+1))
    for split in range(n_splits):
        dataset = PubChem(d.path, n_used = d.n_used, max_nodes=m.max_nodes, 
                          max_edges=m.max_edges, exclude=exclude)
        loader = DataLoader(dataset, batch_size=d.batch_size, shuffle=True, 
                            pin_memory=False, collate_fn=collate_fn)
        trainer.loader = loader
        trainer.fit()
        if trainer.stop_training:
            break
    if trainer.stop_training:
        break

starting epoch 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 624079/624079 [00:56<00:00, 10955.95it/s]
Epoch 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0874:   0%|                                                                                                                        | 0/12482 [00:00<?, ?it/s]
  0%|                                                                                                                                                                                      | 0/200 [00:00<?, ?it/s]
Valid 1: loss 23.044228 0.0000 0.0056 0.0000 0.0000 0.0506:   0%|                                                                                                                          | 0/200 [00:00<?, ?it/s]
Valid 1: loss 23.044228 0.0000 0.0056 0.0000 0.0000 0.0506:   0%|▌                                                                                      

Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0168:  10%|██████████▋                                                                                                      | 19/200 [00:02<00:19,  9.51it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0113:  10%|██████████▋                                                                                                      | 19/200 [00:02<00:19,  9.51it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0113:  10%|███████████▊                                                                                                     | 21/200 [00:02<00:19,  9.05it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0314:  10%|███████████▊                                                                                                     | 21/200 [00:02<00:19,  9.05it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0440:  10%|███████████▊                                                                           

Valid 1: loss 23.044228 0.0056 0.0000 0.0000 0.0000 0.0223:  22%|████████████████████████▎                                                                                        | 43/200 [00:04<00:15,  9.88it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0726:  22%|████████████████████████▎                                                                                        | 43/200 [00:04<00:15,  9.88it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0726:  22%|█████████████████████████▍                                                                                       | 45/200 [00:04<00:15, 10.16it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0345:  22%|█████████████████████████▍                                                                                       | 45/200 [00:05<00:15, 10.16it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0405:  22%|█████████████████████████▍                                                             

Valid 1: loss 23.044228 0.0000 0.0056 0.0000 0.0000 0.0508:  32%|████████████████████████████████████▏                                                                            | 64/200 [00:07<00:15,  8.82it/s]
Valid 1: loss 23.044228 0.0000 0.0056 0.0000 0.0000 0.0508:  33%|█████████████████████████████████████▎                                                                           | 66/200 [00:07<00:14,  9.26it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0659:  33%|█████████████████████████████████████▎                                                                           | 66/200 [00:07<00:14,  9.26it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0301:  33%|█████████████████████████████████████▎                                                                           | 66/200 [00:07<00:14,  9.26it/s]
Valid 1: loss 23.044228 0.0000 0.0000 0.0000 0.0000 0.0301:  34%|██████████████████████████████████████▍                                                

...

Valid 1: loss 8.392073 0.4709 0.5174 0.8256 0.7674 0.6512:  54%|████████████████████████████████████████████████████████████▍                                                    | 107/200 [00:12<00:10,  8.93it/s]
Valid 1: loss 8.392073 0.4709 0.5174 0.8256 0.7674 0.6512:  54%|█████████████████████████████████████████████████████████████                                                    | 108/200 [00:12<00:10,  9.12it/s]
Valid 1: loss 8.392073 0.4625 0.4750 0.8313 0.7750 0.6625:  54%|█████████████████████████████████████████████████████████████                                                    | 108/200 [00:12<00:10,  9.12it/s]
Valid 1: loss 8.392073 0.4625 0.4750 0.8313 0.7750 0.6625:  55%|█████████████████████████████████████████████████████████████▌                                                   | 109/200 [00:12<00:10,  8.61it/s]
Valid 1: loss 8.392073 0.4809 0.5137 0.8197 0.7322 0.6776:  55%|█████████████████████████████████████████████████████████████▌                          

Valid 1: loss 8.392073 0.4011 0.4011 0.7914 0.7701 0.6471:  64%|████████████████████████████████████████████████████████████████████████▎                                        | 128/200 [00:14<00:08,  8.75it/s]
Valid 1: loss 8.392073 0.4719 0.4944 0.7978 0.7697 0.6461:  64%|████████████████████████████████████████████████████████████████████████▎                                        | 128/200 [00:15<00:08,  8.75it/s]
Valid 1: loss 8.392073 0.4719 0.4944 0.7978 0.7697 0.6461:  64%|████████████████████████████████████████████████████████████████████████▉                                        | 129/200 [00:15<00:08,  8.35it/s]
Valid 1: loss 8.392073 0.4695 0.5549 0.8049 0.8049 0.7012:  64%|████████████████████████████████████████████████████████████████████████▉                                        | 129/200 [00:15<00:08,  8.35it/s]
Valid 1: loss 8.392073 0.4695 0.5549 0.8049 0.8049 0.7012:  65%|█████████████████████████████████████████████████████████████████████████▍              

Valid 1: loss 8.392073 0.5090 0.5868 0.8204 0.7605 0.7006:  74%|███████████████████████████████████████████████████████████████████████████████████▌                             | 148/200 [00:17<00:06,  8.37it/s]
Valid 1: loss 8.392073 0.4432 0.5170 0.8239 0.7670 0.6932:  74%|███████████████████████████████████████████████████████████████████████████████████▌                             | 148/200 [00:17<00:06,  8.37it/s]
Valid 1: loss 8.392073 0.4432 0.5170 0.8239 0.7670 0.6932:  74%|████████████████████████████████████████████████████████████████████████████████████▏                            | 149/200 [00:17<00:05,  8.58it/s]
Valid 1: loss 8.392073 0.4250 0.4300 0.8250 0.7950 0.6500:  74%|████████████████████████████████████████████████████████████████████████████████████▏                            | 149/200 [00:17<00:05,  8.58it/s]
Valid 1: loss 8.392073 0.4250 0.4300 0.8250 0.7950 0.6500:  75%|████████████████████████████████████████████████████████████████████████████████████▊   

Valid 1: loss 8.392073 0.3702 0.4641 0.8785 0.7735 0.6961:  84%|██████████████████████████████████████████████████████████████████████████████████████████████▉                  | 168/200 [00:19<00:03,  9.35it/s]
Valid 1: loss 8.392073 0.4176 0.4471 0.7941 0.6529 0.6882:  84%|██████████████████████████████████████████████████████████████████████████████████████████████▉                  | 168/200 [00:19<00:03,  9.35it/s]
Valid 1: loss 8.392073 0.4176 0.4471 0.7941 0.6529 0.6882:  84%|███████████████████████████████████████████████████████████████████████████████████████████████▍                 | 169/200 [00:19<00:03,  9.37it/s]
Valid 1: loss 8.392073 0.4798 0.5029 0.8382 0.7341 0.6936:  84%|███████████████████████████████████████████████████████████████████████████████████████████████▍                 | 169/200 [00:19<00:03,  9.37it/s]
Valid 1: loss 8.392073 0.4798 0.5029 0.8382 0.7341 0.6936:  85%|████████████████████████████████████████████████████████████████████████████████████████

Valid 1: loss 8.392073 0.3678 0.4540 0.8506 0.6954 0.6667:  94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 189/200 [00:21<00:01,  8.71it/s]
Valid 1: loss 8.392073 0.4211 0.4444 0.7778 0.7602 0.6550:  94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 189/200 [00:21<00:01,  8.71it/s]
Valid 1: loss 8.392073 0.4211 0.4444 0.7778 0.7602 0.6550:  95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 190/200 [00:21<00:01,  8.93it/s]
Valid 1: loss 8.392073 0.3908 0.5172 0.8506 0.7529 0.6609:  95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 190/200 [00:22<00:01,  8.93it/s]
Valid 1: loss 8.392073 0.3908 0.5172 0.8506 0.7529 0.6609:  96%|████████████████████████████████████████████████████████████████████████████████████████

In [ ]:
#store config and model
with open(t.es_path+'config.yaml', 'w') as f:
    yaml.dump(config.to_dict(), f, default_flow_style=False)
if name is not None and mode != "offline":
    trained_model_artifact = wandb.Artifact(name, type="model", description="trained selfattn model")
    trained_model_artifact.add_dir(t.es_path)
    run.log_artifact(trained_model_artifact)